# Imports

In [2]:
import polars as pl
import pandas as pd
import datetime
import MetaTrader5 as mt5

# Load data

In [3]:
# Initialize metatrader with python
mt5.initialize()

True

In [4]:
# Define ticker and timetrame
TICKER = "EURUSD"
TICKER_TIMEFRAME = mt5.TIMEFRAME_M5

# Take ticker data
data = pl.DataFrame(
    mt5.copy_rates_range(TICKER, TICKER_TIMEFRAME, datetime.datetime(2023, 5, 20, 20), datetime.datetime(2024, 5, 2))
)

# Convert time ccolumns to datetime
data = data.with_columns(pl.col("time").apply(lambda x: pd.to_datetime(x, unit="s")))

C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_11368\721747910.py:11: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  data = data.with_columns(pl.col("time").apply(lambda x: pd.to_datetime(x, unit="s")))
C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_11368\721747910.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns(pl.col("time").apply(lambda x: pd.to_datetime(x, unit="s")))


# Create the "Y" class to the model

My intention is to make a machine learning model that predicts the price flutuation so I can trade based on it, but it is not that simples, I need to consider how much the price has gonne in both directions, to solve that problem I will make a regression model that predicts a coefficient of fluctuation, for example:

Within the next 10 minutes the price has moved upwards by 5 units and downward by 3 units, so its possible to gain =~ 1.66 units per each unit of risk if trading up and its possible to gain 0.6 units if trading downm downward motions will be resumed by -x while upwards motions will be resumed by +x

In [5]:
# Define time window to look ahead
MINUTES_AHEAD = 10

# sort dataframe
data = data.sort("time")

# Make Y columns
data = (data
    # Return max high and min_low within the next MINUTES_AHEAD window
    .with_columns(
        pl.col("high").reverse().rolling_max(window_size=f"{MINUTES_AHEAD}m", by="time", closed="left").reverse().alias("max_high"),
        pl.col("low").reverse().rolling_min(window_size=f"{MINUTES_AHEAD}m", by="time", closed="left").reverse().alias("min_low")
    )

    # Calculate the max number of units mooved up and mooved down
    .with_columns(
        (pl.col("max_high") - pl.col("close")).alias("max_units_up"),
        (pl.col("close") - pl.col("min_low")).alias("max_units_down")
    )

    # Return 0 if max units down is negative (a.k.a higher than current price)
    .with_columns(max_units_down=
        pl.when(pl.col("max_units_down") < 0)
        .then(0)
        .otherwise(pl.col("max_units_down"))
    )
    # Return 0 if max units up is negative (a.k.a lower than current price)
    .with_columns(max_units_up=
        pl.when(pl.col("max_units_up") < 0)
        .then(0)
        .otherwise(pl.col("max_units_up"))
    )

    # Divide max_units_up per max_units_down and vice-versa to make the "y" columns
    .with_columns(
        (pl.col("max_units_up") / pl.col("max_units_down")).alias("units_up_per_units_down_y"),
        (pl.col("max_units_down") / pl.col("max_units_up")).alias("units_down_per_units_up_y")
    )

    # Resume inf gains as 999 gains, theese are operations where the stop loss would be the same has the last candle clossing price
    .with_columns(units_up_per_units_down_y=
        pl.when(pl.col("units_up_per_units_down_y") > 1_000)
        .then(999)
        .otherwise(pl.col("units_up_per_units_down_y"))
    )
    .with_columns(units_down_per_units_up_y=
        pl.when(pl.col("units_down_per_units_up_y") > 1_000)
        .then(999)
        .otherwise(pl.col("units_down_per_units_up_y"))
    )
)

# Drop auxs columns that generate data leakage and useless columns
data = data.drop(["max_high", "min_low", "max_units_up", "max_units_down", "spread", "real_volume", "tick_volume"])

# Feature engineering

In [18]:
import plotly.graph_objects as go

data_f = data.filter(pl.col("time") > pd.to_datetime("2023-05-22 00:40:00")).head(20)
fig = go.Figure(data=[go.Candlestick(x=data_f["time"],
                open=data_f["open"],
                high=data_f["high"],
                low=data_f["low"],
                close=data_f["close"])])

fig.show()

In [6]:
# Make aux columns to helptechnical analysis candle features
data = (data
    # Make columns that return candle direction
    .with_columns(direction=
        pl.when(
            (pl.col("close") - pl.col("open")) > 0
        )
        .then(pl.lit("up"))
        .when(
            (pl.col("close") - pl.col("open")) < 0
        )
        .then(pl.lit("down"))
        .otherwise(pl.lit("none")
        )
    )
    # Make column to return candle internal and total movement (Direction does not matter here)
    .with_columns(
        (pl.col("close") - pl.col("open")).abs().alias("internal_movement"),
        (pl.col("low") - pl.col("high")).abs().alias("total_movement")
    )

    # Calculate percentage of candle that is filled (Direction does not matter here also)
    .with_columns(
        (pl.col("internal_movement") / pl.col("total_movement")).alias("pct_filled")
    )
)

In [17]:
(data
    # Make three_black_crowns pattern
    .with_columns(three_black_crowns=
        pl.when(
            # 1. 3 last candles where filled
            pl.col("pct_filled") > 0.60,
            pl.col("pct_filled").shift(1) > 0.60,
            pl.col("pct_filled").shift(2) > 0.60,

            # 2. 3 last candles where down?
            pl.col("direction") == "down",
            pl.col("direction").shift(1) == "down",
            pl.col("direction").shift(2) == "down",

            # 3. Last 2 closes where below last 2 lows
            pl.col("close") < pl.col("low").shift(1),
            pl.col("close").shift(1) < pl.col("low").shift(2),
            pl.col("close").shift(2) < pl.col("low").shift(3)
        )
        .then(True)
        .otherwise(False)
    )

    # Make three_white_soldiers pattern
    .with_columns(three_white_soldiers=
        pl.when(
            # 1. 3 last canldes where filled
            pl.col("pct_filled") > 0.60,
            pl.col("pct_filled").shift(1) > 0.60,
            pl.col("pct_filled").shift(2) > 0.60,

            # 2. 3 last candles where up?
            pl.col("direction") == "up",
            pl.col("direction").shift(1) == "up",
            pl.col("direction").shift(2) == "up",

            # 3. Last 2 closes where above last 2 highs
            pl.col("close") > pl.col("high").shift(1),
            pl.col("close").shift(1) > pl.col("high").shift(2),
            pl.col("close").shift(2) > pl.col("high").shift(3)
        )
        .then(True)
        .otherwise(False)
    )

    # Make outside candle pattern
    .with_columns(outside_candle=
        pl.when(
            # Current high and low are "outside" last candle highs and low's
            pl.col("high") > pl.col("high").shift(1),
            pl.col("low") < pl.col("low").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make inside candle pattern
    .with_columns(inside_candle=
        pl.when(
            # Current high and low are "inside" last candle highs and low's
            pl.col("high") < pl.col("high").shift(1),
            pl.col("low") > pl.col("low").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make inside up
    .with_columns(inside_up=
        pl.when(
            # If last candle was and inside candle and current candle close is above last candle high
            pl.col("inside_candle").shift(1),
            pl.col("close") > pl.col("high").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make inside down
    .with_columns(inside_down=
        pl.when(
            # If last candle was and inside candle and current candle close is below last candle low
            pl.col("inside_candle").shift(1),
            pl.col("close") < pl.col("low").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make outside up
    .with_columns(outside_up=
        pl.when(
            # If last candle was and outside candle and current candle close is above last candle high
            pl.col("outside_candle").shift(1),
            pl.col("close") > pl.col("high").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make outside down
    .with_columns(outside_down=
        pl.when(
            # If last candle was and outside candle and current candle close is below last candle low
            pl.col("outside_candle").shift(1),
            pl.col("close") < pl.col("low").shift(1)
        )
        .then(True)
        .otherwise(False)
    )

    # Make lost baby
    .with_columns(lost_baby=
        pl.when(
        )
        .then(True)
        .otherwise(False)
    )

    # Make lost baby high
    .with_columns(lost_baby_high=
        pl.when(
        )
        .then(True)
        .otherwise(False)
    )

        # Make lost baby
    .with_columns(lost_baby_low=
        pl.when(
        )
        .then(True)
        .otherwise(False)
    )
).filter(pl.col("inside_down"))

time,open,high,low,close,units_up_per_units_down_y,units_down_per_units_up_y,direction,internal_movement,total_movement,pct_filled,three_black_crowns,three_white_soldiers,outside_candle,inside_candle,inside_up,inside_down,outside_up,outside_down
datetime[μs],f64,f64,f64,f64,f64,f64,str,f64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool
2023-05-22 00:55:00,1.08062,1.08062,1.08045,1.0805,999.0,0.0,"""down""",0.00012,0.00017,0.705882,false,false,false,false,false,true,false,false
2023-05-22 01:40:00,1.08195,1.08196,1.08171,1.08176,5.0,0.2,"""down""",0.00019,0.00025,0.76,false,false,true,false,false,true,false,false
2023-05-22 02:25:00,1.0823,1.0823,1.08207,1.08209,0.7,1.428571,"""down""",0.00021,0.00023,0.913043,false,false,false,false,false,true,false,false
2023-05-22 03:10:00,1.08192,1.08203,1.08177,1.08185,7.5,0.133333,"""down""",0.00007,0.00026,0.269231,false,false,false,false,false,true,false,false
2023-05-22 08:55:00,1.08141,1.08142,1.08101,1.08104,72.0,0.013889,"""down""",0.00037,0.00041,0.902439,false,false,false,false,false,true,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-04-30 23:55:00,1.06656,1.06662,1.0663,1.06655,1.6,0.625,"""down""",0.00001,0.00032,0.03125,false,false,false,false,false,true,false,false
2024-05-01 05:30:00,1.06593,1.06595,1.06579,1.06582,1.6,0.625,"""down""",0.00011,0.00016,0.6875,false,false,false,false,false,true,false,false
2024-05-01 14:00:00,1.06689,1.06689,1.06671,1.06683,6.2,0.16129,"""down""",0.00006,0.00018,0.333333,false,false,false,false,false,true,false,false
